In [2]:
from selenium import webdriver
from time import sleep

driver = webdriver.Chrome('ChromeDriver/chromedriver')
driver.set_page_load_timeout(2)

try:
    driver.get("https://jewelry.ha.com/c/search-results.zx?N=776+790+231+2040&Ne=2014&erpp=204")
except:
    print("time out")

time out


In [3]:
URL_Links = []
Condition = True

while Condition:
    sleep(5)
    search_result = driver.find_elements_by_class_name('item-title')
    search_result = [link.get_attribute('href') for link in search_result]
    URL_Links.append(search_result)
    try:
        driver.find_element_by_xpath('//*[@id="tabs-1"]/div[3]/a[12]').click()
    except:
        Condition = False


In [6]:
URL_Link = []
for links in URL_Links:
    for link in search_result:
        URL_Link.append(link)
print(len(URL_Link))

205


In [11]:
import pandas as pd
diamond_ring_data_2 = { 'Rings': []}

for link in url:
    try:
        driver.get(link)
        sleep(3)
    except:
        print('TIME OUT')

    sleep(5)
    try:
        ring_price = driver.find_element_by_xpath('//*[@id="page-content"]/div/div[5]/div[1]/div[3]/div/div/small[2]').text
    except:
        ring_price = None
    try:
        description = driver.find_element_by_xpath('//*[@id="auction-description"]/span').text
    except:
        description = None
    try:
        condition_report = driver.find_element_by_xpath('//*[@id="condition-report"]').text
    except:
        condition_report = None

    ring = {
        'Ring Price': [ring_price],
        'Description':[description],
        'Condition Report':[condition_report]
    }

    diamond_ring_data_2['Rings'].append(ring)
    df = pd.DataFrame.from_dict(diamond_ring_data_2)
    df.to_csv('diamond_ring_data_2.csv')


TIME OUT


In [97]:
features_dictionary = {'Carat': [], 'Count': [], 'Price': []}

In [124]:
import pandas as pd

features_dictionary = {'Carat': [], 'Count': [], 'Gross Weight': [], 'Price': []}
data_frame = pd.read_csv('..\Diamond_Rings_data\daimond_ring_data.csv')
data_frame = data_frame.to_dict('records')

for ring in data_frame :
    try:
        mask = ring['Sold Price'].split('FOR: $')[1]
        mask = mask.replace(',', '')
        mask = mask.replace('.', '')
        features_dictionary['Price'].append(float(mask))
    except:
        features_dictionary['Price'].append(None)
    if 'Gem' not in condition_report:
        condition_report = ring['Condition Report'].split('\n')
        for item in condition_report:
            try:
                if 'Count:' in item:
                    features_dictionary['Count'].append(item.split(': ')[1])
            except:
                features_dictionary['Count'].append(None)

    description = ring['Description'].split(',')
    for item in description:
        try:
            if 'Gross weight' in item:
                mask = item.split('Gross weight')[1]
                mask = float(mask.split('grams')[0])
                features_dictionary['Gross Weight'].append(mask)
        except:
            features_dictionary['Gross Weight'].append(None)
        try:
            if 'measuring' in item:
                mask = item.split('measuring')[1]
                mask = mask.split('weighing')[1]
                if 'approximately' in mask:
                    mask = mask.split('approximately')[1]
                if 'respectively' in mask:
                    mask = mask.split('respectively')[1]
                if 'a total of' in mask:
                    mask = mask.split('a total of')[1]
                mask = mask.split('carat')[0]
                features_dictionary['Carat'].append(float(mask))
        except:
            features_dictionary['Carat'].append(None)

    # df = pd.DataFrame.from_dict(features_dictionary)
    # df.to_csv('processed_data.csv')
                    
print(len(features_dictionary['Count']))
print(len(features_dictionary['Carat']))
print(len(features_dictionary['Price']))
print(len(features_dictionary['Gross Weight']))


1778
1440
1445
1405


In [125]:
# pf = pd.DataFrame(data_frame)
# pf